<a href="https://colab.research.google.com/github/Jojowaranyu/weatherwise-Waranyu-B/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---

📄 **Full Assignment Specification**  
See [`ASSIGNMENT.md`](ASSIGNMENT.md) or check the LMS for full details.

📝 **Quick Refresher**  
A one-page summary is available in [`resources/assignment-summary.md`](resources/assignment-summary.md).

---

🧠 **This Notebook Structure is Optional**  
You’re encouraged to reorganise, rename sections, or remove scaffold cells if you prefer — as long as your final version meets the requirements.

✅ You may delete this note before submission.



## 🧰 Setup and Imports

This section imports commonly used packages and installs any additional tools used in the project.

- You may not need all of these unless you're using specific features (e.g. visualisations, advanced prompting).
- The notebook assumes the following packages are **pre-installed** in the provided environment or installable via pip:
  - `requests`, `matplotlib`, `pyinputplus`
  - `fetch-my-weather` (for accessing weather data easily)
  - `hands-on-ai` (for AI logging, comparisons, or prompting tools)

If you're running this notebook in **Google Colab**, uncomment the following lines to install the required packages.


In [18]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install hands-on-ai
!pip install pyinputplus

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyinputplus: filename=pyinputplus-0.2.12-py3-none-any.whl size=11348 sha256=733cdf208e8a434d7e08162dea113570d89c72cda937e80049948b59305b271b
  Stored in directory: /root/.cache/pip/wheels/85/81/c1/325c8199c5c4de8a0b9af900ff9cd8bd69853e3d9419493512
  Created wheel for pysimplevalidate: filename=pysimplevalidate-0.2.12-py3-none-any.whl size=16225 sha256=8f8dd72fe95efa13f8ce73114767ce7a5f01d623a220666e141165d6465bf1e2
  Stored in directory: /root/.cache/pip/wheels/09/de/18/4dd2be0dbdf31870c6f0abde43b0de197b1c095655ad7da5c2
  Created wheel for stdiomask: filename=stdiomask

In [19]:
import os

os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')

Enter your API key: perth


## 📦 Setup and Configuration
Import required packages and setup environment.

In [20]:
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
# ✅ Import after installing (if needed)
from fetch_my_weather import get_weather
from hands_on_ai.chat import get_response

# Add any other setup code here
from typing import TypedDict
from datetime import datetime
from typing import List, Dict, Any

## 🌤️ Weather Data Functions

In [21]:
# Define get_weather_data() function here

class ParsedWeather(TypedDict, total=False):
	location: str
	forecast_days: int
	error: Optional[str]
	current: Dict[str, Any]
	daily: List[Dict[str, Any]]


def _normalize_fmw_json(model_or_dict: Any) -> Dict[str, Any]:
	"""
	Convert fetch_my_weather WeatherResponse (Pydantic) or raw dict to a plain dict.
	This keeps only the keys we care about for the assignment to simplify downstream code.
	"""
	# If it's a pydantic model, use model_dump if available
	if hasattr(model_or_dict, "model_dump"):
		data = model_or_dict.model_dump()  # type: ignore[attr-defined]
	else:
		data = dict(model_or_dict)
	return data

def openWeather_getWeather(city: str, units: str = "metric"):
	api_key = os.getenv("OPENWEATHER_API_KEY")
	if not api_key:
		return None
	try:
		CITY = city
		UNITS = units  # metric for °C or "imperial" for °F
		# Step 1: Call the 5-day forecast API
		url = f"https://api.openweathermap.org/data/2.5/forecast?q={CITY}&appid={api_key}&units={UNITS}"
		response = requests.get(url)
		data = response.json()
		return data
	except Exception as e:
		print(f"Error fetching weather data: {e}")
		return None


def get_weather_data(city: str, forecast_days: int = 5):
	"""
	Retrieve weather data for a specified location.

	Args:
		location (str): City or location name
		forecast_days (int): Number of days to forecast (1-5)

	Returns:
		dict: Weather data including current conditions and forecast
	"""
	try:
		response = openWeather_getWeather(city=city, units="metric")
		return response
	except Exception as e:
		response["error"] = f"Failed to fetch weather data: {e}"
		return response

## 📊 Visualisation Functions

In [22]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here

def _extract_day_labels(daily: List[Dict[str, Any]]) -> List[str]:
        labels: List[str] = []
        for d in daily:
                date_str = d.get("date") or d.get("astronomy", [{}])[0].get("sunrise")
                try:
                        labels.append(datetime.strptime(date_str, "%Y-%m-%d").strftime("%a %d"))
                except Exception:
                        labels.append(str(date_str))
        return labels

def get_temps(data):
        daily_temps = {}

        for item in data["list"]:
                dt_txt = item["dt_txt"] # e.g., '2025-10-16 03:00:00'
                date_str = dt_txt.split(" ")[0] # Just the date part
                temp = item["main"]["temp"]

                if date_str not in daily_temps:
                        daily_temps[date_str] = []

                daily_temps[date_str].append(temp)

        return daily_temps


def create_temperature_visualisation(weather_data: Dict[str, Any], num_days: int = 5, output_type: str = 'display'):
        """
        Create visualisation of temperature data using average daily temperatures.

        Args:
                weather_data (dict): The processed weather data
                num_days (int): The number of days to display in the chart.
                output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

        Returns:
                If output_type is 'figure', returns the matplotlib figure object
                Otherwise, displays the visualisation in the notebook
        """

        # get daily forecast data
        daily = weather_data.get("list", [])
        if not daily:
                print("No daily forecast data available to plot temperatures.")
                return None

        daily_temps = get_temps(weather_data)
        # Slice the data to include only the requested number of days
        sliced_dates = list(daily_temps.keys())[:num_days]
        sliced_temps = [sum(daily_temps[date]) / len(daily_temps[date]) if daily_temps[date] else 0 for date in sliced_dates]


        fig, ax = plt.subplots()
        ax.plot(sliced_dates, sliced_temps, marker='o', label='Average Temp (°C)')
        ax.set_title(f"Daily Average Temperature Forecast for {num_days} Days")
        ax.set_xlabel("Day")
        ax.set_ylabel("Temperature (°C)")
        ax.legend()
        plt.xticks(rotation=45)
        plt.tight_layout()

        if output_type == 'figure':
                return fig
        else:
                plt.show()
                return None


In [23]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
from datetime import datetime
from typing import List, Dict, Any
from collections import defaultdict


def _extract_day_labels(daily: List[Dict[str, Any]]) -> List[str]:
        labels: List[str] = []
        for d in daily:
                date_str = d.get("date") or d.get("astronomy", [{}])[0].get("sunrise")
                try:
                        labels.append(datetime.strptime(date_str, "%Y-%m-%d").strftime("%a %d"))
                except Exception:
                        labels.append(str(date_str))
        return labels

def get_temps(data):
        daily_temps = {}

        for item in data["list"]:
                dt_txt = item["dt_txt"] # e.g., '2025-10-16 03:00:00'
                date_str = dt_txt.split(" ")[0] # Just the date part
                temp = item["main"]["temp"]

                if date_str not in daily_temps:
                        daily_temps[date_str] = []

                daily_temps[date_str].append(temp)

        return daily_temps


def create_temperature_visualisation(weather_data: Dict[str, Any], num_days: int = 5, output_type: str = 'display'):
        """
        Create visualisation of temperature data using average daily temperatures.

        Args:
                weather_data (dict): The processed weather data
                num_days (int): The number of days to display in the chart.
                output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

        Returns:
                If output_type is 'figure', returns the matplotlib figure object
                Otherwise, displays the visualisation in the notebook
        """

        # get daily forecast data
        daily = weather_data.get("list", [])
        if not daily:
                print("No daily forecast data available to plot temperatures.")
                return None

        daily_temps = get_temps(weather_data)
        # Slice the data to include only the requested number of days
        sliced_dates = list(daily_temps.keys())[:num_days]
        sliced_temps = [sum(daily_temps[date]) / len(daily_temps[date]) if daily_temps[date] else 0 for date in sliced_dates]


        fig, ax = plt.subplots()
        ax.plot(sliced_dates, sliced_temps, marker='o', label='Average Temp (°C)')
        ax.set_title(f"Daily Average Temperature Forecast for {num_days} Days")
        ax.set_xlabel("Day")
        ax.set_ylabel("Temperature (°C)")
        ax.legend()
        plt.xticks(rotation=45)
        plt.tight_layout()

        if output_type == 'figure':
                return fig
        else:
                plt.show()
                return None


def create_precipitation_visualisation(weather_data: Dict[str, Any], output_type: str = 'display'):
        """
        Create visualisation of precipitation data.

        Args:
                weather_data (dict): The processed weather data
                output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

        Returns:
                If output_type is 'figure', returns the matplotlib figure object
                Otherwise, displays the visualisation in the notebook
        """
        # OpenWeatherMap provides data in 3-hour intervals under the 'list' key
        forecast_list = weather_data.get("list", [])
        if not forecast_list:
                print("No forecast data available to plot precipitation.")
                return None

        daily_precipitation = defaultdict(float)
        date_labels = []

        for item in forecast_list:
            dt_txt = item["dt_txt"] # e.g., '2025-10-16 03:00:00'
            date_str = dt_txt.split(" ")[0] # Just the date part

            # Add date to labels if not already present (ensures unique daily labels)
            if date_str not in date_labels:
                date_labels.append(date_str)

            # OpenWeatherMap uses 'rain' or 'snow' keys with a '3h' sub-key
            # Total precipitation is the sum of rain and snow in the 3-hour period
            precip_3h = item.get("rain", {}).get("3h", 0) + item.get("snow", {}).get("3h", 0)
            daily_precipitation[date_str] += precip_3h # Accumulate precipitation for the day


        # Sort the daily precipitation data by date
        sorted_dates = sorted(daily_precipitation.keys())
        sorted_precipitation_totals = [daily_precipitation[date] for date in sorted_dates]


        # Format date labels for the plot
        formatted_labels: List[str] = []
        for date_str in sorted_dates:
            try:
                formatted_labels.append(datetime.strptime(date_str, "%Y-%m-%d").strftime("%a %d"))
            except Exception:
                formatted_labels.append(date_str)


        fig, ax = plt.subplots()
        # Limit to the number of days available in the data or a reasonable default (e.g., 5 days)
        num_days_to_plot = min(len(sorted_dates), 5)
        ax.bar(formatted_labels[:num_days_to_plot], sorted_precipitation_totals[:num_days_to_plot], color="#4e79a7")
        ax.set_title("Daily Total Precipitation Forecast")
        ax.set_xlabel("Day")
        ax.set_ylabel("Precipitation (mm)")
        plt.tight_layout()

        if output_type == 'figure':
                return fig
        else:
                plt.show()
                return None

## 🤖 Natural Language Processing

In [24]:
# Define parse_weather_question() and generate_weather_response() here
def parse_weather_question(question):
    """
    Parse a natural language weather question.

    Args:
        question (str): User's weather-related question

    Returns:
        dict: Extracted information including location, time period, and weather attribute
    """
    pass

## 🧭 User Interface

In [25]:
# Define menu functions using pyinputplus or ipywidgets here
import ipywidgets as widgets
from IPython.display import display

# Create input widgets
location_input = widgets.Text(
    value=_LAST_LOCATION if _LAST_LOCATION else '',
    placeholder='Enter location (e.g., Perth)',
    description='Location:',
    disabled=False
)

days_input = widgets.IntText(
    value=_LAST_FORECAST_DAYS,
    description='Days (1-6):',
    disabled=False,
    min=1,
    max=6,
    layout=widgets.Layout(description_width='250 px') # Adjust description width
)

# Create action buttons
button_layout = widgets.Layout(width='250px') # Define a layout for buttons

fetch_button = widgets.Button(description="Fetch Weather Forecast", layout=button_layout)
temp_chart_button = widgets.Button(description="Show Temperature Chart", layout=button_layout)
precip_chart_button = widgets.Button(description="Show Precipitation Chart", layout=button_layout)
should_go_outside_button = widgets.Button(description="Should we go outside tomorrow?", layout=button_layout)
quit_button = widgets.Button(description="Quit", layout=button_layout)

# Create an output widget to display results
output_area = widgets.Output()

# Arrange widgets in a layout
input_widgets = widgets.VBox([location_input, days_input])
button_widgets = widgets.VBox([fetch_button, temp_chart_button, precip_chart_button, should_go_outside_button, quit_button]) # Removed summary_button from layout

ui = widgets.VBox([
    widgets.Label("Greetings! I'm your WeatherWise assistant, ready to bring you the latest forecast."),
    input_widgets,
    button_widgets,
    output_area
])

# Define button click handlers
def on_fetch_button_clicked(b):
    with output_area:
        output_area.clear_output()
        global _LAST_WEATHER, _LAST_LOCATION, _LAST_FORECAST_DAYS # Moved global declaration to the top
        loc = location_input.value
        # Fetch only today's weather for this button
        days = 6  # Fetch 6 days to potentially use for summary or other future features
        data = get_weather_data(loc, forecast_days=days)

        _LAST_WEATHER = data
        _LAST_LOCATION = loc
        _LAST_FORECAST_DAYS = days

        if data and data.get("list"):
            daily_temps = get_temps(data)
            if daily_temps:
                # Get current temperature (using the first available data point as a proxy)
                current_temp = data["list"][0]["main"]["temp"]
                current_description = data["list"][0]["weather"][0]["description"]
                humidity = data["list"][0]["main"]["humidity"]
                wind_speed = data["list"][0]["wind"]["speed"]

                print(f"Weather for {loc}:")
                print(f"Currently: {current_temp:.1f}°C with {current_description}, Humidity: {humidity}%, Wind: {wind_speed} m/s")

                # Get average temperature for the first day (today)
                first_day_date = list(daily_temps.keys())[0]
                temps = daily_temps[first_day_date]
                average_temp = sum(temps) / len(temps) if temps else 0

                print(f"Average temperature for Today ({first_day_date}): {average_temp:.2f}°C")

                # Add conditional messages based on average temperature
                if 28 <= average_temp <= 32:
                    print("Advice: It is quite hot today. Please wear cool clothing and use sunscreen.")
                elif 22 <= average_temp <= 27:
                    print("Advice: Today has good weather.")
                elif 17 <= average_temp <= 21:
                    print("Advice: It is quite cold today. Please wear warm clothes if you go out.")
                elif average_temp < 17:
                    print("Advice: It is freezing! Stay home and turn on the heater.")
                elif average_temp > 32: # Corrected condition for "burning"
                    print("Advice: It is burning hot! Stay home and turn on the air conditioning.")
            else:
                 print("Could not retrieve daily temperature data for the specified location.")


        elif data and data.get("message"):
             print(f"Error fetching weather data: {data['message']}")
        else:
             print("Could not retrieve weather data for the specified location.")


def on_temp_chart_button_clicked(b):
    with output_area:
        output_area.clear_output()
        global _LAST_WEATHER, _LAST_LOCATION, _LAST_FORECAST_DAYS # Moved global declaration to the top
        loc = location_input.value
        if not _LAST_WEATHER or _LAST_LOCATION != loc: # Fetch if no data or location changed
            print(f"Fetching weather data for {loc}...")
            days = 6
            data = get_weather_data(loc, forecast_days=days)
            _LAST_WEATHER = data
            _LAST_LOCATION = loc
            _LAST_FORECAST_DAYS = days
            if not data or data.get("message"):
                print("Could not retrieve weather data. Please check location.")
                return

        if not _LAST_WEATHER or not _LAST_WEATHER.get("list"):
            print("Weather data is not available to create chart. Please fetch weather first.")
            return

        num_days_chart = days_input.value
        create_temperature_visualisation(_LAST_WEATHER, num_days=num_days_chart, output_type='display')

def on_precip_chart_button_clicked(b):
    with output_area:
        output_area.clear_output()
        global _LAST_WEATHER, _LAST_LOCATION, _LAST_FORECAST_DAYS # Moved global declaration to the top
        loc = location_input.value
        if not _LAST_WEATHER or _LAST_LOCATION != loc: # Fetch if no data or location changed
            print(f"Fetching weather data for {loc}...")
            days = 6
            data = get_weather_data(loc, forecast_days=days)
            _LAST_WEATHER = data
            _LAST_LOCATION = loc
            _LAST_FORECAST_DAYS = days
            if not data or data.get("message"):
                print("Could not retrieve weather data. Please check location.")
                return

        if not _LAST_WEATHER or not _LAST_WEATHER.get("list"):
            print("Weather data is not available to create chart. Please fetch weather first.")
            return

        # The precipitation chart currently plots all available days (up to 6)
        create_precipitation_visualisation(_LAST_WEATHER, output_type='display')


def on_should_go_outside_button_clicked(b):
    with output_area:
        output_area.clear_output()
        global _LAST_WEATHER, _LAST_LOCATION, _LAST_FORECAST_DAYS
        loc = location_input.value

        if not _LAST_WEATHER or _LAST_LOCATION != loc: # Fetch if no data or location changed
            print(f"Fetching weather data for {loc}...")
            days = 6
            data = get_weather_data(loc, forecast_days=days)
            _LAST_WEATHER = data
            _LAST_LOCATION = loc
            _LAST_FORECAST_DAYS = days
            if not data or data.get("message"):
                print("Could not retrieve weather data. Please check location.")
                return

        if not _LAST_WEATHER or not _LAST_WEATHER.get("list"):
            print("Weather data is not available for tomorrow's forecast. Please fetch weather first.")
            return

        # Get tomorrow's data - assuming the first day in the list is today, the second is tomorrow
        if len(_LAST_WEATHER.get("list", [])) > 8: # OpenWeatherMap provides data in 3-hour intervals, so need at least 8*3 = 24 hours for tomorrow
            tomorrow_data_points = _LAST_WEATHER["list"][8:16] # Assuming data points 8-15 cover tomorrow
            if tomorrow_data_points:
                # Calculate average temperature for tomorrow
                tomorrow_temps = [item["main"]["temp"] for item in tomorrow_data_points]
                average_temp_tomorrow = sum(tomorrow_temps) / len(tomorrow_temps) if tomorrow_temps else 0

                # Calculate total precipitation for tomorrow (summing up 3-hour intervals)
                # OpenWeatherMap's forecast doesn't always have 'rain' or 'snow' keys, need to check
                tomorrow_precipitation = sum(item.get("rain", {}).get("3h", 0) + item.get("snow", {}).get("3h", 0) for item in tomorrow_data_points)

                # Get wind speed and description for tomorrow (using the first data point for simplicity)
                tomorrow_wind_speed = tomorrow_data_points[0]["wind"]["speed"]
                tomorrow_weather_description = tomorrow_data_points[0]["weather"][0]["description"]

                tomorrow_date = tomorrow_data_points[0]["dt_txt"].split(" ")[0]

                print(f"Tomorrow's Weather ({tomorrow_date}) for {loc}:")
                print(f"Average Temperature: {average_temp_tomorrow:.2f}°C")
                print(f"Total Precipitation: {tomorrow_precipitation:.2f} mm")
                print(f"Wind Speed: {tomorrow_wind_speed:.2f} m/s")
                print(f"Conditions: {tomorrow_weather_description}")


                # Provide advice based on tomorrow's weather
                print("\nShould we go outside tomorrow?")
                if tomorrow_precipitation > 0.5: # Threshold for significant rain
                    print("Advice: It looks like there will be significant rain tomorrow. It might be best to stay inside or be prepared for wet conditions.")
                elif average_temp_tomorrow < 10: # Threshold for cold
                     print("Advice: It will be quite cold tomorrow. Bundle up if you go outside!")
                elif average_temp_tomorrow > 30: # Threshold for hot
                     print("Advice: It will be quite hot tomorrow. Stay hydrated and seek shade if you go outside.")
                else:
                    print("Advice: The weather looks good for going outside tomorrow!")

            else:
                print("Could not retrieve detailed forecast data for tomorrow.")

        else:
             print("Insufficient forecast data to determine tomorrow's weather.")


def on_quit_button_clicked(b):
    with output_area:
        output_area.clear_output()
        print("Goodbye!")
    # In a real application, you might want to stop execution here
    # For a notebook, simply printing a message is sufficient


# Link buttons to handlers
fetch_button.on_click(on_fetch_button_clicked)
temp_chart_button.on_click(on_temp_chart_button_clicked)
# summary_button.on_click(on_summary_button_clicked) # Removed
precip_chart_button.on_click(on_precip_chart_button_clicked)
should_go_outside_button.on_click(on_should_go_outside_button_clicked)
quit_button.on_click(on_quit_button_clicked)


# Display the UI
display(ui)

## 🧩 Main Application Logic

In [26]:
# Tie everything together here
def generate_weather_response(parsed_question, weather_data):
    """
    Generate a natural language response to a weather question.

    Args:
        parsed_question (dict): Parsed question data
        weather_data (dict): Weather data

    Returns:
        str: Natural language response
    """
    pass

## 🧪 Testing and Examples

In [27]:
# Include sample input/output for each function

## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.

# Task
Integrate an "Ask Question" button into the WeatherWise assistant UI that uses AI to parse user questions and generate natural language weather responses.

## Add "ask question" button to the ui

### Subtask:
Modify the existing UI code to include a new button for asking weather questions.


**Reasoning**:
The subtask requires modifying the existing UI code to add a new button and a text input for asking questions. This involves creating the new widgets and adding them to the appropriate layout containers within the existing UI structure. I will use the existing code block for the UI and add the new widgets there.



In [28]:
# Define menu functions using pyinputplus or ipywidgets here
import ipywidgets as widgets
from IPython.display import display

# Global variables to store last fetched data (optional, for convenience)
_LAST_WEATHER = None
_LAST_LOCATION = ''
_LAST_FORECAST_DAYS = 6 # Default to 6 days forecast for comprehensive data


# Create input widgets
location_input = widgets.Text(
    value=_LAST_LOCATION if _LAST_LOCATION else '',
    placeholder='Enter location (e.g., Perth)',
    description='Location:',
    disabled=False
)

days_input = widgets.IntText(
    value=_LAST_FORECAST_DAYS,
    description='Days (1-6):',
    disabled=False,
    min=1,
    max=6,
    layout=widgets.Layout(description_width='250 px') # Adjust description width
)

# Create action buttons
button_layout = widgets.Layout(width='250px') # Define a layout for buttons

fetch_button = widgets.Button(description="Fetch Weather Forecast", layout=button_layout)
temp_chart_button = widgets.Button(description="Show Temperature Chart", layout=button_layout)
precip_chart_button = widgets.Button(description="Show Precipitation Chart", layout=button_layout)
should_go_outside_button = widgets.Button(description="Should we go outside tomorrow?", layout=button_layout)
quit_button = widgets.Button(description="Quit", layout=button_layout)

# New widgets for asking questions
question_input = widgets.Text(
    value='',
    placeholder='Ask a weather question (e.g., What is the weather like today?)',
    description='Your Question:',
    disabled=False,
    layout=widgets.Layout(width='500px') # Adjust width for question input
)

ask_question_button = widgets.Button(description="Ask Question", layout=button_layout)


# Create an output widget to display results
output_area = widgets.Output()

# Arrange widgets in a layout
input_widgets = widgets.VBox([location_input, days_input])
# Add the new question input and ask button to the button widgets layout
button_widgets = widgets.VBox([fetch_button, temp_chart_button, precip_chart_button, should_go_outside_button, ask_question_button, quit_button]) # Removed summary_button from layout

ui = widgets.VBox([
    widgets.Label("Greetings! I'm your WeatherWise assistant, ready to bring you the latest forecast."),
    input_widgets,
    widgets.VBox([question_input, ask_question_button]), # Group question input and ask button
    button_widgets,
    output_area
])

# Define button click handlers
def on_fetch_button_clicked(b):
    with output_area:
        output_area.clear_output()
        global _LAST_WEATHER, _LAST_LOCATION, _LAST_FORECAST_DAYS # Moved global declaration to the top
        loc = location_input.value
        # Fetch only today's weather for this button
        days = 6  # Fetch 6 days to potentially use for summary or other future features
        data = get_weather_data(loc, forecast_days=days)

        _LAST_WEATHER = data
        _LAST_LOCATION = loc
        _LAST_FORECAST_DAYS = days

        if data and data.get("list"):
            daily_temps = get_temps(data)
            if daily_temps:
                # Get current temperature (using the first available data point as a proxy)
                current_temp = data["list"][0]["main"]["temp"]
                current_description = data["list"][0]["weather"][0]["description"]
                humidity = data["list"][0]["main"]["humidity"]
                wind_speed = data["list"][0]["wind"]["speed"]

                print(f"Weather for {loc}:")
                print(f"Currently: {current_temp:.1f}°C with {current_description}, Humidity: {humidity}%, Wind: {wind_speed} m/s")

                # Get average temperature for the first day (today)
                first_day_date = list(daily_temps.keys())[0]
                temps = daily_temps[first_day_date]
                average_temp = sum(temps) / len(temps) if temps else 0

                print(f"Average temperature for Today ({first_day_date}): {average_temp:.2f}°C")

                # Add conditional messages based on average temperature
                if 28 <= average_temp <= 32:
                    print("Advice: It is quite hot today. Please wear cool clothing and use sunscreen.")
                elif 22 <= average_temp <= 27:
                    print("Advice: Today has good weather.")
                elif 17 <= average_temp <= 21:
                    print("Advice: It is quite cold today. Please wear warm clothes if you go out.")
                elif average_temp < 17:
                    print("Advice: It is freezing! Stay home and turn on the heater.")
                elif average_temp > 32: # Corrected condition for "burning"
                    print("Advice: It is burning hot! Stay home and turn on the air conditioning.")
            else:
                 print("Could not retrieve daily temperature data for the specified location.")


        elif data and data.get("message"):
             print(f"Error fetching weather data: {data['message']}")
        else:
             print("Could not retrieve weather data for the specified location.")


def on_temp_chart_button_clicked(b):
    with output_area:
        output_area.clear_output()
        global _LAST_WEATHER, _LAST_LOCATION, _LAST_FORECAST_DAYS # Moved global declaration to the top
        loc = location_input.value
        if not _LAST_WEATHER or _LAST_LOCATION != loc: # Fetch if no data or location changed
            print(f"Fetching weather data for {loc}...")
            days = 6
            data = get_weather_data(loc, forecast_days=days)
            _LAST_WEATHER = data
            _LAST_LOCATION = loc
            _LAST_FORECAST_DAYS = days
            if not data or data.get("message"):
                print("Could not retrieve weather data. Please check location.")
                return

        if not _LAST_WEATHER or not _LAST_WEATHER.get("list"):
            print("Weather data is not available to create chart. Please fetch weather first.")
            return

        num_days_chart = days_input.value
        create_temperature_visualisation(_LAST_WEATHER, num_days=num_days_chart, output_type='display')

def on_precip_chart_button_clicked(b):
    with output_area:
        output_area.clear_output()
        global _LAST_WEATHER, _LAST_LOCATION, _LAST_FORECAST_DAYS # Moved global declaration to the top
        loc = location_input.value
        if not _LAST_WEATHER or _LAST_LOCATION != loc: # Fetch if no data or location changed
            print(f"Fetching weather data for {loc}...")
            days = 6
            data = get_weather_data(loc, forecast_days=days)
            _LAST_WEATHER = data
            _LAST_LOCATION = loc
            _LAST_FORECAST_DAYS = days
            if not data or data.get("message"):
                print("Could not retrieve weather data. Please check location.")
                return

        if not _LAST_WEATHER or not _LAST_WEATHER.get("list"):
            print("Weather data is not available to create chart. Please fetch weather first.")
            return

        # The precipitation chart currently plots all available days (up to 6)
        create_precipitation_visualisation(_LAST_WEATHER, output_type='display')


def on_should_go_outside_button_clicked(b):
    with output_area:
        output_area.clear_output()
        global _LAST_WEATHER, _LAST_LOCATION, _LAST_FORECAST_DAYS
        loc = location_input.value

        if not _LAST_WEATHER or _LAST_LOCATION != loc: # Fetch if no data or location changed
            print(f"Fetching weather data for {loc}...")
            days = 6
            data = get_weather_data(loc, forecast_days=days)
            _LAST_WEATHER = data
            _LAST_LOCATION = loc
            _LAST_FORECAST_DAYS = days
            if not data or data.get("message"):
                print("Could not retrieve weather data. Please check location.")
                return

        if not _LAST_WEATHER or not _LAST_WEATHER.get("list"):
            print("Weather data is not available for tomorrow's forecast. Please fetch weather first.")
            return

        # Get tomorrow's data - assuming the first day in the list is today, the second is tomorrow
        if len(_LAST_WEATHER.get("list", [])) > 8: # OpenWeatherMap provides data in 3-hour intervals, so need at least 8*3 = 24 hours for tomorrow
            tomorrow_data_points = _LAST_WEATHER["list"][8:16] # Assuming data points 8-15 cover tomorrow
            if tomorrow_data_points:
                # Calculate average temperature for tomorrow
                tomorrow_temps = [item["main"]["temp"] for item in tomorrow_data_points]
                average_temp_tomorrow = sum(tomorrow_temps) / len(tomorrow_temps) if tomorrow_temps else 0

                # Calculate total precipitation for tomorrow (summing up 3-hour intervals)
                # OpenWeatherMap's forecast doesn't always have 'rain' or 'snow' keys, need to check
                tomorrow_precipitation = sum(item.get("rain", {}).get("3h", 0) + item.get("snow", {}).get("3h", 0) for item in tomorrow_data_points)

                # Get wind speed and description for tomorrow (using the first data point for simplicity)
                tomorrow_wind_speed = tomorrow_data_points[0]["wind"]["speed"]
                tomorrow_weather_description = tomorrow_data_points[0]["weather"][0]["description"]

                tomorrow_date = tomorrow_data_points[0]["dt_txt"].split(" ")[0]

                print(f"Tomorrow's Weather ({tomorrow_date}) for {loc}:")
                print(f"Average Temperature: {average_temp_tomorrow:.2f}°C")
                print(f"Total Precipitation: {tomorrow_precipitation:.2f} mm")
                print(f"Wind Speed: {tomorrow_wind_speed:.2f} m/s")
                print(f"Conditions: {tomorrow_weather_description}")


                # Provide advice based on tomorrow's weather
                print("\nShould we go outside tomorrow?")
                if tomorrow_precipitation > 0.5: # Threshold for significant rain
                    print("Advice: It looks like there will be significant rain tomorrow. It might be best to stay inside or be prepared for wet conditions.")
                elif average_temp_tomorrow < 10: # Threshold for cold
                     print("Advice: It will be quite cold tomorrow. Bundle up if you go outside!")
                elif average_temp_tomorrow > 30: # Threshold for hot
                     print("Advice: It will be quite hot tomorrow. Stay hydrated and seek shade if you go outside.")
                else:
                    print("Advice: The weather looks good for going outside tomorrow!")

            else:
                print("Could not retrieve detailed forecast data for tomorrow.")

        else:
             print("Insufficient forecast data to determine tomorrow's weather.")

def on_ask_question_button_clicked(b):
    """Handler for the 'Ask Question' button."""
    with output_area:
        output_area.clear_output()
        question = question_input.value
        loc = location_input.value

        if not question:
            print("Please enter a question.")
            return

        if not loc:
            print("Please enter a location.")
            return

        # For now, just print the question and location.
        # In the next steps, we will integrate the AI parsing and response generation.
        print(f"You asked: '{question}' about weather in {loc}")
        print("Processing your question...")
        # Placeholder for AI parsing and response generation logic


def on_quit_button_clicked(b):
    with output_area:
        output_area.clear_output()
        print("Goodbye!")
    # In a real application, you might want to stop execution here
    # For a notebook, simply printing a message is sufficient


# Link buttons to handlers
fetch_button.on_click(on_fetch_button_clicked)
temp_chart_button.on_click(on_temp_chart_button_clicked)
# summary_button.on_click(on_summary_button_clicked) # Removed
precip_chart_button.on_click(on_precip_chart_button_clicked)
should_go_outside_button.on_click(on_should_go_outside_button_clicked)
ask_question_button.on_click(on_ask_question_button_clicked) # Link the new button
quit_button.on_click(on_quit_button_clicked)


# Display the UI
display(ui)

## Implement `parse weather question`

### Subtask:
Write code to parse the user's natural language question using an AI model to extract key information like location, time period, and weather attribute.


**Reasoning**:
Write the code for the `parse_weather_question` function to use the AI model to extract information from the user's question.



In [29]:
import json
from typing import Optional

def parse_weather_question(question: str) -> Dict[str, Optional[str]]:
    """
    Parse a natural language weather question using an AI model.

    Args:
        question (str): User's weather-related question

    Returns:
        dict: Extracted information including location, time period, and weather attribute.
              Returns None for fields that couldn't be extracted.
    """
    prompt = f"""Analyze the following weather question and extract the key information:
- Location (city name)
- Time period (e.g., today, tomorrow, next 3 days, specific date if mentioned)
- Weather attribute (e.g., temperature, precipitation, wind, conditions)

Return the extracted information as a JSON string with keys 'location', 'time_period', and 'weather_attribute'. If a piece of information is not explicitly mentioned, return null for that key.

Question: "{question}"
JSON Output:
"""

    try:
        # Assuming get_response is available and works with the configured AI model
        ai_response = get_response(prompt)
        # The AI is instructed to return a JSON string directly
        parsed_data = json.loads(ai_response.strip())

        # Ensure the expected keys are present, even if their values are null
        return {
            'location': parsed_data.get('location'),
            'time_period': parsed_data.get('time_period'),
            'weather_attribute': parsed_data.get('weather_attribute')
        }

    except Exception as e:
        print(f"Error parsing AI response: {e}")
        # Return a dictionary with None values in case of error
        return {
            'location': None,
            'time_period': None,
            'weather_attribute': None
        }


## Implement `generate weather response`

### Subtask:
Write code to generate a natural language response based on the parsed question and available weather data using an AI model.


**Reasoning**:
Define the `generate_weather_response` function to construct a prompt for the AI model using the parsed question and weather data, then call the AI to get the response.



In [30]:
def generate_weather_response(parsed_question: Dict[str, Optional[str]], weather_data: Dict[str, Any]) -> str:
    """
    Generate a natural language response based on the parsed question
    and available weather data using an AI model.

    Args:
        parsed_question (dict): Parsed question data including location, time_period, weather_attribute.
        weather_data (dict): Weather data retrieved for the location.

    Returns:
        str: Natural language response generated by the AI.
             Returns an informative message if data is insufficient or AI call fails.
    """
    location = parsed_question.get('location')
    time_period = parsed_question.get('time_period')
    weather_attribute = parsed_question.get('weather_attribute')

    if not weather_data or not weather_data.get("list"):
        return "I don't have weather data available to answer that question. Please fetch the weather data first."

    # Construct a simplified summary of the relevant weather data for the AI
    # This part needs refinement based on how time_period and weather_attribute are parsed
    # For now, let's provide a general summary of the forecast data.
    weather_summary = "Weather data available:\n"
    for i, item in enumerate(weather_data["list"]):
        dt_txt = item["dt_txt"]
        temp = item["main"]["temp"]
        description = item["weather"][0]["description"]
        precip = item.get("rain", {}).get("3h", 0) + item.get("snow", {}).get("3h", 0)
        weather_summary += f"  Time: {dt_txt}, Temp: {temp}°C, Conditions: {description}, Precipitation: {precip} mm\n"
        if i >= 15: # Limit the summary to a reasonable amount of data points (e.g., 2 days * 8 intervals/day)
            break


    prompt = f"""You are a weather assistant. Based on the following weather data and the user's question, generate a concise and natural language response. Only use the information provided in the weather data. If the data does not contain enough information to answer the question, state that you cannot answer based on the available data.

User's question was about:
Location: {location if location else 'not specified'}
Time Period: {time_period if time_period else 'not specified, assume near future/forecast'}
Weather Attribute: {weather_attribute if weather_attribute else 'not specified'}

Weather Data:
{weather_summary}

Based on the above, answer the user's question in a natural language sentence or two:
"""

    try:
        ai_response = get_response(prompt)
        return ai_response.strip()
    except Exception as e:
        print(f"Error getting response from AI: {e}")
        return "I apologize, but I encountered an error while trying to generate a response."


## Integrate "ask question" button logic

### Subtask:
Add a handler function for the new button that calls `parse_weather_question`, fetches necessary weather data, calls `generate_weather_response`, and displays the AI-generated response in the output area.


**Reasoning**:
The subtask is to add the logic to the `on_ask_question_button_clicked` function to parse the question, fetch weather data, generate an AI response, and display it. This involves calling the previously defined `parse_weather_question` and `generate_weather_response` functions and integrating them within the existing button handler.



In [31]:
# Define menu functions using pyinputplus or ipywidgets here
import ipywidgets as widgets
from IPython.display import display

# Global variables to store last fetched data (optional, for convenience)
_LAST_WEATHER = None
_LAST_LOCATION = ''
_LAST_FORECAST_DAYS = 6 # Default to 6 days forecast for comprehensive data


# Create input widgets
location_input = widgets.Text(
    value=_LAST_LOCATION if _LAST_LOCATION else '',
    placeholder='Enter location (e.g., Perth)',
    description='Location:',
    disabled=False
)

days_input = widgets.IntText(
    value=_LAST_FORECAST_DAYS,
    description='Days (1-6):',
    disabled=False,
    min=1,
    max=6,
    layout=widgets.Layout(description_width='250 px') # Adjust description width
)

# Create action buttons
button_layout = widgets.Layout(width='250px') # Define a layout for buttons

fetch_button = widgets.Button(description="Fetch Weather Forecast", layout=button_layout)
temp_chart_button = widgets.Button(description="Show Temperature Chart", layout=button_layout)
precip_chart_button = widgets.Button(description="Show Precipitation Chart", layout=button_layout)
should_go_outside_button = widgets.Button(description="Should we go outside tomorrow?", layout=button_layout)
quit_button = widgets.Button(description="Quit", layout=button_layout)

# New widgets for asking questions
question_input = widgets.Text(
    value='',
    placeholder='Ask a weather question (e.g., What is the weather like today?)',
    description='Your Question:',
    disabled=False,
    layout=widgets.Layout(width='500px') # Adjust width for question input
)

ask_question_button = widgets.Button(description="Ask Question", layout=button_layout)


# Create an output widget to display results
output_area = widgets.Output()

# Arrange widgets in a layout
input_widgets = widgets.VBox([location_input, days_input])
# Add the new question input and ask button to the button widgets layout
button_widgets = widgets.VBox([fetch_button, temp_chart_button, precip_chart_button, should_go_outside_button, ask_question_button, quit_button]) # Removed summary_button from layout

ui = widgets.VBox([
    widgets.Label("Greetings! I'm your WeatherWise assistant, ready to bring you the latest forecast."),
    input_widgets,
    widgets.VBox([question_input, ask_question_button]), # Group question input and ask button
    button_widgets,
    output_area
])

# Define button click handlers
def on_fetch_button_clicked(b):
    with output_area:
        output_area.clear_output()
        global _LAST_WEATHER, _LAST_LOCATION, _LAST_FORECAST_DAYS # Moved global declaration to the top
        loc = location_input.value
        # Fetch only today's weather for this button
        days = 6  # Fetch 6 days to potentially use for summary or other future features
        data = get_weather_data(loc, forecast_days=days)

        _LAST_WEATHER = data
        _LAST_LOCATION = loc
        _LAST_FORECAST_DAYS = days

        if data and data.get("list"):
            daily_temps = get_temps(data)
            if daily_temps:
                # Get current temperature (using the first available data point as a proxy)
                current_temp = data["list"][0]["main"]["temp"]
                current_description = data["list"][0]["weather"][0]["description"]
                humidity = data["list"][0]["main"]["humidity"]
                wind_speed = data["list"][0]["wind"]["speed"]

                print(f"Weather for {loc}:")
                print(f"Currently: {current_temp:.1f}°C with {current_description}, Humidity: {humidity}%, Wind: {wind_speed} m/s")

                # Get average temperature for the first day (today)
                first_day_date = list(daily_temps.keys())[0]
                temps = daily_temps[first_day_date]
                average_temp = sum(temps) / len(temps) if temps else 0

                print(f"Average temperature for Today ({first_day_date}): {average_temp:.2f}°C")

                # Add conditional messages based on average temperature
                if 28 <= average_temp <= 32:
                    print("Advice: It is quite hot today. Please wear cool clothing and use sunscreen.")
                elif 22 <= average_temp <= 27:
                    print("Advice: Today has good weather.")
                elif 17 <= average_temp <= 21:
                    print("Advice: It is quite cold today. Please wear warm clothes if you go out.")
                elif average_temp < 17:
                    print("Advice: It is freezing! Stay home and turn on the heater.")
                elif average_temp > 32: # Corrected condition for "burning"
                    print("Advice: It is burning hot! Stay home and turn on the air conditioning.")
            else:
                 print("Could not retrieve daily temperature data for the specified location.")


        elif data and data.get("message"):
             print(f"Error fetching weather data: {data['message']}")
        else:
             print("Could not retrieve weather data for the specified location.")


def on_temp_chart_button_clicked(b):
    with output_area:
        output_area.clear_output()
        global _LAST_WEATHER, _LAST_LOCATION, _LAST_FORECAST_DAYS # Moved global declaration to the top
        loc = location_input.value
        if not _LAST_WEATHER or _LAST_LOCATION != loc: # Fetch if no data or location changed
            print(f"Fetching weather data for {loc}...")
            days = 6
            data = get_weather_data(loc, forecast_days=days)
            _LAST_WEATHER = data
            _LAST_LOCATION = loc
            _LAST_FORECAST_DAYS = days
            if not data or data.get("message"):
                print("Could not retrieve weather data. Please check location.")
                return

        if not _LAST_WEATHER or not _LAST_WEATHER.get("list"):
            print("Weather data is not available to create chart. Please fetch weather first.")
            return

        num_days_chart = days_input.value
        create_temperature_visualisation(_LAST_WEATHER, num_days=num_days_chart, output_type='display')

def on_precip_chart_button_clicked(b):
    with output_area:
        output_area.clear_output()
        global _LAST_WEATHER, _LAST_LOCATION, _LAST_FORECAST_DAYS # Moved global declaration to the top
        loc = location_input.value
        if not _LAST_WEATHER or _LAST_LOCATION != loc: # Fetch if no data or location changed
            print(f"Fetching weather data for {loc}...")
            days = 6
            data = get_weather_data(loc, forecast_days=days)
            _LAST_WEATHER = data
            _LAST_LOCATION = loc
            _LAST_FORECAST_DAYS = days
            if not data or data.get("message"):
                print("Could not retrieve weather data. Please check location.")
                return

        if not _LAST_WEATHER or not _LAST_WEATHER.get("list"):
            print("Weather data is not available to create chart. Please fetch weather first.")
            return

        # The precipitation chart currently plots all available days (up to 6)
        create_precipitation_visualisation(_LAST_WEATHER, output_type='display')


def on_should_go_outside_button_clicked(b):
    with output_area:
        output_area.clear_output()
        global _LAST_WEATHER, _LAST_LOCATION, _LAST_FORECAST_DAYS
        loc = location_input.value

        if not _LAST_WEATHER or _LAST_LOCATION != loc: # Fetch if no data or location changed
            print(f"Fetching weather data for {loc}...")
            days = 6
            data = get_weather_data(loc, forecast_days=days)
            _LAST_WEATHER = data
            _LAST_LOCATION = loc
            _LAST_FORECAST_DAYS = days
            if not data or data.get("message"):
                print("Could not retrieve weather data. Please check location.")
                return

        if not _LAST_WEATHER or not _LAST_WEATHER.get("list"):
            print("Weather data is not available for tomorrow's forecast. Please fetch weather first.")
            return

        # Get tomorrow's data - assuming the first day in the list is today, the second is tomorrow
        if len(_LAST_WEATHER.get("list", [])) > 8: # OpenWeatherMap provides data in 3-hour intervals, so need at least 8*3 = 24 hours for tomorrow
            tomorrow_data_points = _LAST_WEATHER["list"][8:16] # Assuming data points 8-15 cover tomorrow
            if tomorrow_data_points:
                # Calculate average temperature for tomorrow
                tomorrow_temps = [item["main"]["temp"] for item in tomorrow_data_points]
                average_temp_tomorrow = sum(tomorrow_temps) / len(tomorrow_temps) if tomorrow_temps else 0

                # Calculate total precipitation for tomorrow (summing up 3-hour intervals)
                # OpenWeatherMap's forecast doesn't always have 'rain' or 'snow' keys, need to check
                tomorrow_precipitation = sum(item.get("rain", {}).get("3h", 0) + item.get("snow", {}).get("3h", 0) for item in tomorrow_data_points)

                # Get wind speed and description for tomorrow (using the first data point for simplicity)
                tomorrow_wind_speed = tomorrow_data_points[0]["wind"]["speed"]
                tomorrow_weather_description = tomorrow_data_points[0]["weather"][0]["description"]

                tomorrow_date = tomorrow_data_points[0]["dt_txt"].split(" ")[0]

                print(f"Tomorrow's Weather ({tomorrow_date}) for {loc}:")
                print(f"Average Temperature: {average_temp_tomorrow:.2f}°C")
                print(f"Total Precipitation: {tomorrow_precipitation:.2f} mm")
                print(f"Wind Speed: {tomorrow_wind_speed:.2f} m/s")
                print(f"Conditions: {tomorrow_weather_description}")


                # Provide advice based on tomorrow's weather
                print("\nShould we go outside tomorrow?")
                if tomorrow_precipitation > 0.5: # Threshold for significant rain
                    print("Advice: It looks like there will be significant rain tomorrow. It might be best to stay inside or be prepared for wet conditions.")
                elif average_temp_tomorrow < 10: # Threshold for cold
                     print("Advice: It will be quite cold tomorrow. Bundle up if you go outside!")
                elif average_temp_tomorrow > 30: # Threshold for hot
                     print("Advice: It will be quite hot tomorrow. Stay hydrated and seek shade if you go outside.")
                else:
                    print("Advice: The weather looks good for going outside tomorrow!")

            else:
                print("Could not retrieve detailed forecast data for tomorrow.")

        else:
             print("Insufficient forecast data to determine tomorrow's weather.")

def on_ask_question_button_clicked(b):
    """Handler for the 'Ask Question' button."""
    with output_area:
        output_area.clear_output()
        question = question_input.value
        loc = location_input.value # Get location from input field as fallback

        if not question:
            print("Please enter a question.")
            return

        print("Processing your question...")

        # 1. Call parse_weather_question
        parsed_data = parse_weather_question(question)
        print(f"Parsed data: {parsed_data}") # Temporary print for debugging

        # 3. Check for location
        extracted_location = parsed_data.get('location')
        effective_location = extracted_location if extracted_location else loc

        if not effective_location:
            print("Please specify a location in your question or enter it in the Location field.")
            return

        print(f"Using location: {effective_location}") # Temporary print for debugging

        # 4. Fetch weather data
        # Fetch weather data for a fixed number of days (e.g., 6 days) for comprehensive context
        days_to_fetch = 6
        print(f"Fetching weather data for {effective_location} for {days_to_fetch} days...") # Temporary print for debugging
        weather_data = get_weather_data(effective_location, forecast_days=days_to_fetch)

        # 5. Check if weather data was retrieved
        if not weather_data or weather_data.get("message"):
            print(f"Could not retrieve weather data for {effective_location}. Please check the location name.")
            return

        print("Weather data fetched successfully.") # Temporary print for debugging

        # 6. Call generate_weather_response
        print("Generating AI response...") # Temporary print for debugging
        ai_response = generate_weather_response(parsed_data, weather_data)

        # 7. Print the AI response
        output_area.clear_output() # Clear output before displaying final response
        print(ai_response)


def on_quit_button_clicked(b):
    with output_area:
        output_area.clear_output()
        print("Goodbye!")
    # In a real application, you might want to stop execution here
    # For a notebook, simply printing a message is sufficient


# Link buttons to handlers
fetch_button.on_click(on_fetch_button_clicked)
temp_chart_button.on_click(on_temp_chart_button_clicked)
# summary_button.on_click(on_summary_button_clicked) # Removed
precip_chart_button.on_click(on_precip_chart_button_clicked)
should_go_outside_button.on_click(on_should_go_outside_button_clicked)
ask_question_button.on_click(on_ask_question_button_clicked) # Link the new button
quit_button.on_click(on_quit_button_clicked)


# Display the UI
display(ui)

**Reasoning**:
The previous step successfully updated the `on_ask_question_button_clicked` function to include calls to `parse_weather_question`, `get_weather_data`, and `generate_weather_response`. It also included temporary print statements for debugging. The next step is to remove these temporary print statements as per the instructions.



In [35]:
# Define menu functions using pyinputplus or ipywidgets here
import ipywidgets as widgets
from IPython.display import display

# Global variables to store last fetched data (optional, for convenience)
_LAST_WEATHER = None
_LAST_LOCATION = ''
_LAST_FORECAST_DAYS = 6 # Default to 6 days forecast for comprehensive data


# Create input widgets
location_input = widgets.Text(
    value=_LAST_LOCATION if _LAST_LOCATION else '',
    placeholder='Enter location (e.g., Perth)',
    description='Location:',
    disabled=False
)

days_input = widgets.IntText(
    value=_LAST_FORECAST_DAYS,
    description='Days (1-6):',
    disabled=False,
    min=1,
    max=6,
    layout=widgets.Layout(description_width='250 px') # Adjust description width
)

# Create action buttons
button_layout = widgets.Layout(width='250px') # Define a layout for buttons

fetch_button = widgets.Button(description="Fetch Weather Forecast", layout=button_layout)
temp_chart_button = widgets.Button(description="Show Temperature Chart", layout=button_layout)
precip_chart_button = widgets.Button(description="Show Precipitation Chart", layout=button_layout)
should_go_outside_button = widgets.Button(description="Should we go outside tomorrow?", layout=button_layout)
quit_button = widgets.Button(description="Quit", layout=button_layout)

# New widgets for asking questions
question_input = widgets.Text(
    value='',
    placeholder='Ask a weather question (e.g., What is the weather like today?)',
    description='Your Question:',
    disabled=False,
    layout=widgets.Layout(width='500px') # Adjust width for question input
)

ask_question_button = widgets.Button(description="Ask Question", layout=button_layout)


# Create an output widget to display results
output_area = widgets.Output()

# Arrange widgets in a layout
input_widgets = widgets.VBox([location_input, days_input])
# Add the new question input and ask button to the button widgets layout
button_widgets = widgets.VBox([fetch_button, temp_chart_button, precip_chart_button, should_go_outside_button, ask_question_button, quit_button]) # Removed summary_button from layout

ui = widgets.VBox([
    widgets.Label("Greetings! I'm your WeatherWise assistant, ready to bring you the latest forecast."),
    input_widgets,
    widgets.VBox([question_input, ask_question_button]), # Group question input and ask button
    button_widgets,
    output_area
])

# Define button click handlers
def on_fetch_button_clicked(b):
    with output_area:
        output_area.clear_output()
        global _LAST_WEATHER, _LAST_LOCATION, _LAST_FORECAST_DAYS # Moved global declaration to the top
        loc = location_input.value
        # Fetch only today's weather for this button
        days = 6  # Fetch 6 days to potentially use for summary or other future features
        data = get_weather_data(loc, forecast_days=days)

        _LAST_WEATHER = data
        _LAST_LOCATION = loc
        _LAST_FORECAST_DAYS = days

        if data and data.get("list"):
            daily_temps = get_temps(data)
            if daily_temps:
                # Get current temperature (using the first available data point as a proxy)
                current_temp = data["list"][0]["main"]["temp"]
                current_description = data["list"][0]["weather"][0]["description"]
                humidity = data["list"][0]["main"]["humidity"]
                wind_speed = data["list"][0]["wind"]["speed"]

                print(f"Weather for {loc}:")
                print(f"Currently: {current_temp:.1f}°C with {current_description}, Humidity: {humidity}%, Wind: {wind_speed} m/s")

                # Get average temperature for the first day (today)
                first_day_date = list(daily_temps.keys())[0]
                temps = daily_temps[first_day_date]
                average_temp = sum(temps) / len(temps) if temps else 0

                print(f"Average temperature for Today ({first_day_date}): {average_temp:.2f}°C")

                # Add conditional messages based on average temperature
                if 28 <= average_temp <= 32:
                    print("Advice: It is quite hot today. Please wear cool clothing and use sunscreen.")
                elif 22 <= average_temp <= 27:
                    print("Advice: Today has good weather.")
                elif 17 <= average_temp <= 21:
                    print("Advice: It is quite cold today. Please wear warm clothes if you go out.")
                elif average_temp < 17:
                    print("Advice: It is freezing! Stay home and turn on the heater.")
                elif average_temp > 32: # Corrected condition for "burning"
                    print("Advice: It is burning hot! Stay home and turn on the air conditioning.")
            else:
                 print("Could not retrieve daily temperature data for the specified location.")


        elif data and data.get("message"):
             print(f"Error fetching weather data: {data['message']}")
        else:
             print("Could not retrieve weather data for the specified location.")


def on_temp_chart_button_clicked(b):
    with output_area:
        output_area.clear_output()
        global _LAST_WEATHER, _LAST_LOCATION, _LAST_FORECAST_DAYS # Moved global declaration to the top
        loc = location_input.value
        if not _LAST_WEATHER or _LAST_LOCATION != loc: # Fetch if no data or location changed
            print(f"Fetching weather data for {loc}...")
            days = 6
            data = get_weather_data(loc, forecast_days=days)
            _LAST_WEATHER = data
            _LAST_LOCATION = loc
            _LAST_FORECAST_DAYS = days
            if not data or data.get("message"):
                print("Could not retrieve weather data. Please check location.")
                return

        if not _LAST_WEATHER or not _LAST_WEATHER.get("list"):
            print("Weather data is not available to create chart. Please fetch weather first.")
            return

        num_days_chart = days_input.value
        create_temperature_visualisation(_LAST_WEATHER, num_days=num_days_chart, output_type='display')

def on_precip_chart_button_clicked(b):
    with output_area:
        output_area.clear_output()
        global _LAST_WEATHER, _LAST_LOCATION, _LAST_FORECAST_DAYS # Moved global declaration to the top
        loc = location_input.value
        if not _LAST_WEATHER or _LAST_LOCATION != loc: # Fetch if no data or location changed
            print(f"Fetching weather data for {loc}...")
            days = 6
            data = get_weather_data(loc, forecast_days=days)
            _LAST_WEATHER = data
            _LAST_LOCATION = loc
            _LAST_FORECAST_DAYS = days
            if not data or data.get("message"):
                print("Could not retrieve weather data. Please check location.")
                return

        if not _LAST_WEATHER or not _LAST_WEATHER.get("list"):
            print("Weather data is not available to create chart. Please fetch weather first.")
            return

        # The precipitation chart currently plots all available days (up to 6)
        create_precipitation_visualisation(_LAST_WEATHER, output_type='display')


def on_should_go_outside_button_clicked(b):
    with output_area:
        output_area.clear_output()
        global _LAST_WEATHER, _LAST_LOCATION, _LAST_FORECAST_DAYS
        loc = location_input.value

        if not _LAST_WEATHER or _LAST_LOCATION != loc: # Fetch if no data or location changed
            print(f"Fetching weather data for {loc}...")
            days = 6
            data = get_weather_data(loc, forecast_days=days)
            _LAST_WEATHER = data
            _LAST_LOCATION = loc
            _LAST_FORECAST_DAYS = days
            if not data or data.get("message"):
                print("Could not retrieve weather data. Please check location.")
                return

        if not _LAST_WEATHER or not _LAST_WEATHER.get("list"):
            print("Weather data is not available for tomorrow's forecast. Please fetch weather first.")
            return

        # Get tomorrow's data - assuming the first day in the list is today, the second is tomorrow
        if len(_LAST_WEATHER.get("list", [])) > 8: # OpenWeatherMap provides data in 3-hour intervals, so need at least 8*3 = 24 hours for tomorrow
            tomorrow_data_points = _LAST_WEATHER["list"][8:16] # Assuming data points 8-15 cover tomorrow
            if tomorrow_data_points:
                # Calculate average temperature for tomorrow
                tomorrow_temps = [item["main"]["temp"] for item in tomorrow_data_points]
                average_temp_tomorrow = sum(tomorrow_temps) / len(tomorrow_temps) if tomorrow_temps else 0

                # Calculate total precipitation for tomorrow (summing up 3-hour intervals)
                # OpenWeatherMap's forecast doesn't always have 'rain' or 'snow' keys, need to check
                tomorrow_precipitation = sum(item.get("rain", {}).get("3h", 0) + item.get("snow", {}).get("3h", 0) for item in tomorrow_data_points)

                # Get wind speed and description for tomorrow (using the first data point for simplicity)
                tomorrow_wind_speed = tomorrow_data_points[0]["wind"]["speed"]
                tomorrow_weather_description = tomorrow_data_points[0]["weather"][0]["description"]

                tomorrow_date = tomorrow_data_points[0]["dt_txt"].split(" ")[0]

                print(f"Tomorrow's Weather ({tomorrow_date}) for {loc}:")
                print(f"Average Temperature: {average_temp_tomorrow:.2f}°C")
                print(f"Total Precipitation: {tomorrow_precipitation:.2f} mm")
                print(f"Wind Speed: {tomorrow_wind_speed:.2f} m/s")
                print(f"Conditions: {tomorrow_weather_description}")


                # Provide advice based on tomorrow's weather
                print("\nShould we go outside tomorrow?")
                if tomorrow_precipitation > 0.5: # Threshold for significant rain
                    print("Advice: It looks like there will be significant rain tomorrow. It might be best to stay inside or be prepared for wet conditions.")
                elif average_temp_tomorrow < 10: # Threshold for cold
                     print("Advice: It will be quite cold tomorrow. Bundle up if you go outside!")
                elif average_temp_tomorrow > 30: # Threshold for hot
                     print("Advice: It will be quite hot tomorrow. Stay hydrated and seek shade if you go outside.")
                else:
                    print("Advice: The weather looks good for going outside tomorrow!")

            else:
                print("Could not retrieve detailed forecast data for tomorrow.")

        else:
             print("Insufficient forecast data to determine tomorrow's weather.")

def on_ask_question_button_clicked(b):
    """Handler for the 'Ask Question' button."""
    with output_area:
        output_area.clear_output()
        question = question_input.value
        loc = location_input.value # Get location from input field as fallback

        if not question:
            print("Please enter a question.")
            return

        # 1. Call parse_weather_question
        parsed_data = parse_weather_question(question)

        # 3. Check for location
        extracted_location = parsed_data.get('location')
        effective_location = extracted_location if extracted_location else loc

        if not effective_location:
            print("Please specify a location in your question or enter it in the Location field.")
            return


        # 4. Fetch weather data
        # Fetch weather data for a fixed number of days (e.g., 6 days) for comprehensive context
        days_to_fetch = 6
        weather_data = get_weather_data(effective_location, forecast_days=days_to_fetch)

        # 5. Check if weather data was retrieved
        if not weather_data or weather_data.get("message"):
            print(f"Could not retrieve weather data for {effective_location}. Please check the location name.")
            return


        # 6. Call generate_weather_response
        ai_response = generate_weather_response(parsed_data, weather_data)

        # 7. Print the AI response
        output_area.clear_output() # Clear output before displaying final response
        print(ai_response)


def on_quit_button_clicked(b):
    with output_area:
        output_area.clear_output()
        print("Goodbye!")
    # In a real application, you might want to stop execution here
    # For a notebook, simply printing a message is sufficient


# Link buttons to handlers
fetch_button.on_click(on_fetch_button_clicked)
temp_chart_button.on_click(on_temp_chart_button_clicked)
# summary_button.on_click(on_summary_button_clicked) # Removed
precip_chart_button.on_click(on_precip_chart_button_clicked)
should_go_outside_button.on_click(on_should_go_outside_button_clicked)
ask_question_button.on_click(on_ask_question_button_clicked) # Link the new button
quit_button.on_click(on_quit_button_clicked)


# Display the UI
display(ui)

In [34]:
import os

print(os.getenv('HANDS_ON_AI_SERVER'))

http://ollama.serveur.au


## Summary:

### Data Analysis Key Findings

*   The WeatherWise assistant UI was successfully updated to include a text input field for user questions and an "Ask Question" button.
*   A function `parse_weather_question` was implemented to use an AI model to extract key information (location, time period, weather attribute) from a natural language question, returning the results in a JSON format.
*   A function `generate_weather_response` was created to take the parsed question and available weather data, construct a prompt for an AI model, and generate a natural language response based on the data.
*   The handler for the "Ask Question" button was updated to orchestrate the process: it retrieves the user's question, calls `parse_weather_question`, determines the effective location (from the question or input field), fetches 6 days of weather data using `get_weather_data`, calls `generate_weather_response` with the parsed data and fetched weather, and displays the AI's response in the output area.
*   Error handling was included for cases where no question or location is provided, or if weather data fetching fails.

### Insights or Next Steps

*   Further refinement of the `parse_weather_question` and `generate_weather_response` functions is needed to improve AI accuracy in understanding user intent and generating more contextually relevant responses based on the available weather data.
*   Consider adding more specific data filtering within `generate_weather_response` based on the parsed `time_period` and `weather_attribute` to provide more precise information to the AI and potentially improve the quality of the generated response.
